In [34]:
from lrg.prompting.loader import prompt_loader
from jinja2 import Environment, nodes

dummy = prompt_loader.get_or_select_template("dummy_prompt.md")

In [48]:
dummy.filename

'/app/lrg/prompting/templates/dummy_prompt.md'

In [49]:
text = open(dummy.filename, "r").read()
text

'{{name}} is a {{folder}} path.'

In [52]:
env = Environment()
ast = env.parse(text)
ast

Template(body=[Output(nodes=[Name(name='name', ctx='load'), TemplateData(data=' is a '), Name(name='folder', ctx='load'), TemplateData(data=' path.')])])

In [53]:
from jinja2 import meta
meta.find_undeclared_variables(ast)

{'folder', 'name'}

In [67]:
from jinja2 import Environment, nodes, meta

def extract_all_variables(template_string):
    """Extracts all variables including nested attributes from a Jinja template."""

    env = Environment()
    parsed_content = env.parse(template_string)
    undeclared_vars = meta.find_undeclared_variables(parsed_content)

    all_variables = set()
    for node in parsed_content.find_all(nodes.Getattr):
        full_var = node.attr
        current_obj = node.node
        while isinstance(current_obj, nodes.Getattr):
            full_var = f"{current_obj.attr}.{full_var}"
            current_obj = current_obj.node
        if isinstance(current_obj, nodes.Name):
            full_var = f"{current_obj.name}.{full_var}"
        all_variables.add(full_var)

    return all_variables

template_string = "{{name.path}} is a {{name.folder}} path."
extracted_variables = extract_all_variables(template_string)
print(extracted_variables)  # Output: {'name.folder', 'name.path'} 

{'name.path', 'name.folder'}


In [65]:
from jinja2 import Environment, nodes
# Create a Jinja2 environment.
env = Environment()
template_string = "{{name.path}} is a {{name.folder}} path."
# Find all variables used in the template.
parsed_content = env.parse(template_string)
meta.find_undeclared_variables(parsed_content)
# variables = set(n.name for n in parsed_content.find_all(nodes.Name))
# missing_variables = variables - set(context.keys())
# if missing_variables:
#     raise ValueError(f"Missing required variables: {', '.join(missing_variables)}")

{'name'}

In [64]:
parsed_content

Template(body=[Output(nodes=[Getattr(node=Name(name='name', ctx='load'), attr='path', ctx='load'), TemplateData(data=' is a '), Getattr(node=Name(name='name', ctx='load'), attr='folder', ctx='load'), TemplateData(data=' path.')])])

In [60]:
variables

{'folder', 'name'}

In [1]:
from pydantic import BaseModel
from openai import OpenAI

class ReasoningStep(BaseModel):
    explanation: str
    action: str

class LegalReasoning(BaseModel):
    steps: list[ReasoningStep]

client = OpenAI()

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are a helpful math tutor."},
        {"role": "user", "content": "solve 8x + 31 = 2"},
    ],
    response_format=LegalReasoning,
)

message = completion.choices[0].message
if message.parsed:
    print(message.parsed.steps)
    print(message.parsed.final_answer)
else:
    print(message.refusal)

[Step(explanation='We start with the equation: 8x + 31 = 2.', output='8x + 31 = 2'), Step(explanation='Subtract 31 from both sides to isolate the term with x.', output='8x = 2 - 31'), Step(explanation='Calculate the subtraction: 2 - 31 = -29.', output='8x = -29'), Step(explanation='Divide both sides by 8 to solve for x.', output='x = -29/8')]
x = -29/8


In [4]:
message

ParsedChatCompletionMessage[MathResponse](content='{"steps":[{"explanation":"We start with the equation: 8x + 31 = 2.","output":"8x + 31 = 2"},{"explanation":"Subtract 31 from both sides to isolate the term with x.","output":"8x = 2 - 31"},{"explanation":"Calculate the subtraction: 2 - 31 = -29.","output":"8x = -29"},{"explanation":"Divide both sides by 8 to solve for x.","output":"x = -29/8"}],"final_answer":"x = -29/8"}', refusal=None, role='assistant', function_call=None, tool_calls=[], parsed=MathResponse(steps=[Step(explanation='We start with the equation: 8x + 31 = 2.', output='8x + 31 = 2'), Step(explanation='Subtract 31 from both sides to isolate the term with x.', output='8x = 2 - 31'), Step(explanation='Calculate the subtraction: 2 - 31 = -29.', output='8x = -29'), Step(explanation='Divide both sides by 8 to solve for x.', output='x = -29/8')], final_answer='x = -29/8'))

In [10]:
message.parsed.steps[0]

Step(explanation='We start with the equation: 8x + 31 = 2.', output='8x + 31 = 2')